In [12]:
import pandas as pd
from gurobipy import *
import itertools

In [4]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {'delay': 500,'code_cells_only': True}})

{'highlight_selected_word': {'delay': 500, 'code_cells_only': True},
 'load_extensions': {'highlight_selected_word/main': True}}

In [25]:
# disponibilidad(1) corresponde a las fechas de marzo
data_raw = pd.read_csv('disponibilidad(1).csv',encoding="ISO-8859-1",sep=";")
data_marzo = pd.read_csv('marzo.csv',encoding="ISO-8859-1",sep=";")
data_abril = pd.read_csv('abril.csv',encoding="ISO-8859-1",sep=";")
data_mayo = pd.read_csv('mayo.csv',encoding="ISO-8859-1",sep=";")
data_junio = pd.read_csv('junio.csv',encoding="ISO-8859-1",sep=";")
data_total = pd.read_csv('dispo_marzo_junio.csv',encoding="ISO-8859-1",sep=";")

#dejar columna Name como index
data = data_raw.set_index('nombre')
data_marzo = data_marzo.set_index('Name')
data_abril = data_abril.set_index('Name')
data_mayo = data_mayo.set_index('Name')
data_junio = data_junio.set_index('Name')
data_total = data_total.set_index('Name')

# cambiar nombres de columnas por mes-dia
data_marzo.columns = ['mar-'+col for col in data_marzo.columns]
data_marzo.columns = [col.replace(" ","") for col in data_marzo.columns]
data_abril.columns = ['abr-'+col for col in data_abril.columns]
data_abril.columns = [col.replace(" ","") for col in data_abril.columns]
data_mayo.columns = ['may-'+col for col in data_mayo.columns]
data_mayo.columns = [col.replace(" ","") for col in data_mayo.columns]
data_junio.columns = ['jun-'+col for col in data_junio.columns]
data_junio.columns = [col.replace(" ","") for col in data_junio.columns]

# concatenar dataframes
data_concat = pd.concat([data_marzo, data_abril, data_mayo,data_junio], axis=1)
data_concat.fillna('no',inplace=True)
data_total.fillna('no', inplace=True)
data.fillna('no',inplace=True)

# poner todo en minuscula
data_concat = data_concat.apply(lambda x: x.astype(str).str.lower())
data_total = data_total.apply(lambda x: x.astype(str).str.lower())

# algunos conjuntos utiles
months = ['mar','abr','may','jun']
keys = ['mar-11','mar-18','mar-25','abr-1','abr-8','abr-15','abr-22','abr-29','may-6','may-13','may-20','may-27','jun-3','jun-10','jun-17','jun-24']
weeks = {}
for key in keys:
    mes = key.split('-')[0]
    dia = int(key.split('-')[1])
    weeks[key] = [mes+'-'+str(dia+j) for j in range(5)]

# lo anterior supone semanas incluidas
# en un solo mes, arreglamos abr-29
weeks['abr-29'][2] = 'may-1'
weeks['abr-29'][3] = 'may-2'
weeks['abr-29'][4] = 'may-3'
lugares = {'SPENCE','MEL','CC'}
nombres = set(data_concat.index)
dias = list(data_concat)

In [17]:
# definir variables
x = {}
for t in dias:
    for n in nombres:
        for l in lugares:
            x[t,n,l] = scheduler.addVar(vtype=GRB.BINARY, name = "x%s,%s,%s" % (t,n,l))

Index(['mar-1', 'mar-2', 'mar-3', 'mar-4', 'mar-5', 'mar-6', 'mar-7', 'mar-8',
       'mar-9', 'mar-10',
       ...
       'jun-19', 'jun-20', 'jun-21', 'jun-22', 'jun-23', 'jun-24', 'jun-25',
       'jun-26', 'jun-27', 'jun-28'],
      dtype='object', length=120)

In [47]:
scheduler = Model()

# definir variables
x = {}
for t in dias:
    for n in nombres:
        for l in lugares:
            x[t,n,l] = scheduler.addVar(vtype=GRB.BINARY, name = "x%s,%s,%s" % (t,n,l))
            

scheduler.update()
# construir fn objetivo
expr = LinExpr()
dicc_import_lugar = {'SPENCE':2,'MEL':100,'CC':1}
for lugar in lugares:
    expr += LinExpr([dicc_import_lugar[lugar] for t,n in itertools.product(dias,nombres)],
                    [x[t,n,lugar] for t,n in itertools.product(dias,nombres)])

# restricciones de disponibilidad
for n in nombres:
    for t in dias:
        for l in lugares:
            ans = data_concat.at[n,t]
            if ans[0]=='s' or ans[0]=='y':
                scheduler.addConstr(x[t,n,l]<= 1)
            if ans[0]=='n':
                scheduler.addConstr(x[t,n,l]<=0)
            
# solo se puede estar
# en un lugar a la vez
for t in dias:
    for n in nombres:
        scheduler.addConstr(sum([x[t,n,l] for l in lugares]) <= 1)

# desplazarse una vez 
# a la semana
for key in keys:
    for n in nombres:
        scheduler.addConstr(sum([x[t,n,l] for l,t in itertools.product(lugares,weeks[key])]) <= 1)
        
# MEL al menos con 3
# personas
for key in keys:
    scheduler.addConstr(sum([x[t,n,'MEL'] for t,n in itertools.product(weeks[key],nombres)]) == 3)

# analogo para SPENCE y CC
# pero con dos
for key in keys:
    scheduler.addConstr(sum([x[t,n,'SPENCE'] for t,n in itertools.product(weeks[key],nombres)]) >= 1)
#    scheduler.addConstr(sum([x[t,n,'CC'] for t,n in itertools.product(weeks[key],nombres)]) >= 2)
    
# en general, que no se queden
# dos semanas consecutivas en el 
# mismo lugar
for i in range(len(keys)-1):
    for n in ['Marcos','Paula','Ed','Cecilia','Robert','Jorge']:
        for l in lugares:
            scheduler.addConstr(sum([x[t,n,l] for t in weeks[keys[i+1]]])<= 1-sum([x[t,n,l] for t in weeks[keys[i]]]))
            
scheduler.setObjective(expr,GRB.MAXIMIZE)
scheduler.optimize()

Optimize a model with 9230 rows, 6480 columns and 22860 nonzeros
Variable types: 0 continuous, 6480 integer (6480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 4927.0000000
Presolve removed 9012 rows and 6303 columns
Presolve time: 0.03s
Presolved: 218 rows, 177 columns, 558 nonzeros
Found heuristic solution: objective 8425.0000000
Variable types: 0 continuous, 177 integer (171 binary)

Root relaxation: objective 8.432000e+03, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8432.0000000 8432.00000  0.00%     -    0s

Explored 0 nodes (51 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 8432 8425 8378 

Optimal solutio

In [48]:
# resumen de los viajes por semana
print('Personas-Destinos')
for key in keys:
    print('\n\nSemana del %s ' % key)
    for lugar in lugares:
        for n in nombres:
            if sum([x[t,n,lugar].x for t in weeks[key]])==1:
                print('%s viaja a %s' % (n,lugar))

Personas-Destinos


Semana del mar-11 
Cecilia viaja a MEL
Robert viaja a MEL
Ed viaja a MEL
Marcos viaja a SPENCE
María viaja a SPENCE
Jorge viaja a SPENCE
Edgar viaja a SPENCE
Cata viaja a SPENCE


Semana del mar-18 
Marcos viaja a MEL
Jorge viaja a MEL
Silvana viaja a MEL
Cecilia viaja a SPENCE
María viaja a SPENCE
Robert viaja a SPENCE
Edgar viaja a SPENCE
Ed viaja a SPENCE


Semana del mar-25 
Cecilia viaja a MEL
Robert viaja a MEL
Mauricio viaja a MEL
Marcos viaja a SPENCE
María viaja a SPENCE
Edgar viaja a SPENCE
Paula viaja a SPENCE
Gustavo viaja a SPENCE
Cata viaja a SPENCE


Semana del abr-1 
Marcos viaja a MEL
Silvana viaja a MEL
Ed viaja a MEL
Cecilia viaja a SPENCE
María viaja a SPENCE
Robert viaja a SPENCE
Jorge viaja a SPENCE
Edgar viaja a SPENCE
Mauricio viaja a SPENCE


Semana del abr-8 
Cecilia viaja a MEL
Robert viaja a MEL
Jorge viaja a MEL
Marcos viaja a SPENCE
María viaja a SPENCE
Edgar viaja a SPENCE
Ed viaja a SPENCE
Paula viaja a SPENCE
Mauricio viaja a SPENCE


In [ ]:
# algunos frecuencias

# cuantas personas diferentes del
# equipo fueron cada mes a los diferentes
# destinos


In [33]:
data_concat = data_concat.apply(lambda x: x.astype(str).str.lower())
data_total = data_total.apply(lambda x: x.astype(str).str.lower())

In [22]:
('abr-31', 'Cecilia', 'CC') in x.keys()

False

In [23]:
'abr-31' in dias

False

In [35]:
data_total.columns = [col.replace(" ","") for col in data_total.columns]
data_total.columns

Index(['11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       ...
       '19.3', '20.3', '21.3', '22.3', '23.3', '24.3', '25.3', '26.3', '27.3',
       '28.3'],
      dtype='object', length=110)

In [56]:
toy = data['4'].loc[data.index=='Paula']

In [62]:
# recuperar valor dado el index y la columna
data.at['Paula','4']

'no'

In [7]:
data.index

Index(['Paula', 'Douglas', 'Jariel', 'Claudia', 'Marcos', 'Mauricio',
       'Juan Pablo', 'Robert', 'Silvana', 'Mar­a', 'Gustavo', 'Ed', 'Jorge',
       'Cecilia', 'Cata', 'Simon', 'Edgar', 'Constanza'],
      dtype='object', name='nombre')

In [16]:
# recuperamos los indices para el modelo
# (i.e. dias, lugares,nombres)
dias = set(data.columns)
lugares = {'SPENCE','MEL','CC'}
nombres = set(data.index)
semanas = {}
# creamos un diccionario
# dia:dias de esa semana
for i in [4,11,18,25]:
    semanas[str(i)] = [str(i+j) for j in range(5)]
key_semanas = list(semanas.keys())

In [17]:
for semana in key_semanas:
    print(semanas[semana])

['4', '5', '6', '7', '8']
['11', '12', '13', '14', '15']
['18', '19', '20', '21', '22']
['25', '26', '27', '28', '29']


In [14]:
m=Model()
x = {}
for t in dias:
    for n in nombres:
        for l in lugares:
            x[t,n,l] = m.addVar(vtype=GRB.BINARY, name = "x%s,%s,%s" % (t,n,l))
m.update()

expr = LinExpr()
dicc_import_lugar = {'SPENCE':2,'MEL':100,'CC':1}
for lugar in lugares:
    expr += LinExpr([dicc_import_lugar[lugar] for t,n in itertools.product(dias,nombres)],
                    [x[t,n,lugar] for t,n in itertools.product(dias,nombres)])
#expr = LinExpr([1 for t,n,l in itertools.product(dias,nombres,lugares)],
#               [x[t,n,l] for t,n,l in itertools.product(dias,nombres,lugares)])

# restricciones de disponibilidad
for n in nombres:
    for t in dias:
        for l in lugares:
            if data.at[n,t]=='yes':
                m.addConstr(x[t,n,l]<= 1)
            else:
                m.addConstr(x[t,n,l]<=0)
            
# restricciones fisicas
# una persona solo puede estar en un lugar por dia
for t in dias:
    for n in nombres:
        m.addConstr(sum([x[t,n,l] for l in lugares])<= 1)
        
# Una persona viaja a lo mas una vez
# por semana
for semana in key_semanas:
    for n in nombres:
        m.addConstr(sum([x[t,n,l] for t,l in itertools.product(semanas[semana],lugares)])<= 1)
        
# cada semana todos los lugares son visitados
# por al menos dos personas
for semana in key_semanas:
    for l in ['SPENCE','CC']:
        m.addConstr(sum([x[t,n,l] for t,n in itertools.product(semanas[semana],nombres)])>=2)
# lo mismo que lo anterior, pero con 
# cota inferior igual a 3
# restar el caso Mauricio en MEL
for semana in key_semanas:
    for l in ['MEL']:
        m.addConstr(sum([x[t,n,l] for t,n in itertools.product(semanas[semana],nombres) if not n=='Mauricio'])>=3)
        
# no viajar al mismo lugar en semanas consecutivas
for i in range(3):
    for n in ['Marcos','Paula','Ed']:
        for l in lugares:
            m.addConstr(sum([x[t,n,l] for t in semanas[key_semanas[i+1]]])<=1-sum([x[t,n,l] for t in semanas[key_semanas[i]]]))
    
m.setObjective(expr, GRB.MAXIMIZE)
m.Params.presolve = 0
m.optimize()

Academic license - for non-commercial use only
Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 1983 rows, 1404 columns and 5218 nonzeros
Variable types: 0 continuous, 1404 integer (1404 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 1226.0000000
Variable types: 0 continuous, 1404 integer (1404 binary)

Root relaxation: objective 1.324000e+03, 165 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1324.0000000 1324.00000  0.00%     -    0s

Explored 0 nodes (165 simplex iterations) in 0.05 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 1324 1226 

Optimal solution found (tolerance 1.00e-04)
Best

In [ ]:
# resumen de los viajes por semana
print('Personas-Destinos')
for semana in key_semanas:
    print('\n\nSemana del %s de marzo' % semana)
    for lugar in lugares:
        for n in nombres:
            if sum([x[t,n,lugar].x for t in semanas[semana]])==1:
                print('%s viaja a %s' % (n,lugar))

In [208]:
name = 'Edgar'
lugar = 'MEL'
for semana in key_semanas:
    print(sum([x[t,name,lugar].x for t in semanas[semana]]))

1.0
1.0
1.0
1.0


In [143]:
# cuantas personas viajan a SPENCE la primera semana
weekone = key_semanas[3]
print(sum([x[t,n,lugar].x for t,n in itertools.product(semanas[weekone],nombres)]))

for n in nombres:
    if sum([x[t,n,lugar].x for t in semanas[weekone]])==1:
        print('%s viaja a %s' % (n,lugar))

2.0
Cata viaja a CC
Mauricio viaja a CC


In [148]:
weekone = key_semanas[0]
lugar = 'CC'
for n in nombres:
    if sum([x[t,n,lugar].x for t in semanas[weekone]])==1:
        print('%s viaja a %s' % (n,lugar))

Silvana viaja a CC
Mar­a viaja a CC


In [15]:
# resumen de los viajes por semana
print('Personas-Destinos')
for semana in key_semanas:
    print('\n\nSemana del %s de marzo' % semana)
    for lugar in lugares:
        for n in nombres:
            if sum([x[t,n,lugar].x for t in semanas[semana]])==1:
                print('%s viaja a %s' % (n,lugar))


Personas-Destinos


Semana del 4 de marzo
Robert viaja a SPENCE
Edgar viaja a SPENCE
Mar­a viaja a CC
Marcos viaja a CC
Silvana viaja a MEL
Jorge viaja a MEL
Ed viaja a MEL


Semana del 11 de marzo
Jorge viaja a SPENCE
Marcos viaja a SPENCE
Ed viaja a CC
Cata viaja a CC
Robert viaja a MEL
Mar­a viaja a MEL
Edgar viaja a MEL


Semana del 18 de marzo
Mar­a viaja a SPENCE
Jorge viaja a SPENCE
Robert viaja a CC
Silvana viaja a CC
Ed viaja a MEL
Marcos viaja a MEL
Edgar viaja a MEL


Semana del 25 de marzo
Robert viaja a SPENCE
Mar­a viaja a SPENCE
Mauricio viaja a CC
Marcos viaja a CC
Gustavo viaja a MEL
Cata viaja a MEL
Paula viaja a MEL
Edgar viaja a MEL


In [210]:
# numero de personas diferentes que fueron al asset
# en el mes
var_by_place = {}
for l in lugares:
    var_by_place[l] = sum([x[t,n,l].x for t,n in itertools.product(dias,nombres)])

In [211]:
var_by_place

{'SPENCE': 8.0, 'MEL': 13.0, 'CC': 8.0}